In [1]:
import time

import numpy as np
import pandas as pd
import seaborn as sns
sns.set_style('white')
import statsmodels.api as sm
from sklearn import preprocessing 
import ema_workbench.em_framework.evaluators


import os
from dest_directories import gz_path, fig_path
from dicemodel.noDICE_v5 import PyDICE
model_version = 'v5'



In [2]:
from ema_workbench import (perform_experiments, Model, Policy, Scenario, ReplicatorModel, RealParameter, IntegerParameter, TimeSeriesOutcome, ScalarOutcome, ArrayOutcome, Constant, ema_logging, SequentialEvaluator, MultiprocessingEvaluator, IpyparallelEvaluator)
from ema_workbench import save_results, load_results
from ema_workbench.analysis import prim, cart
from ema_workbench.analysis import scenario_discovery_util as sdutil
from ema_workbench.analysis import plotting, plotting_util

ema_logging.log_to_stderr(ema_logging.INFO)

model = PyDICE()
dice_sm = Model('dicesmEMA', function=model)

dice_sm.uncertainties = [IntegerParameter('t2xco2_index', 0, 999),
                #  IntegerParameter('t2xco2_dist',0,2),
                #  IntegerParameter('fdamage', 0, 2),
                    RealParameter('tfp_gr',  0.07, 0.09),
                    RealParameter('sigma_gr', -0.012, -0.008),
                    RealParameter('pop_gr', 0.1, 0.15),
                    RealParameter('fosslim',  4000.0, 13649),
                    IntegerParameter('cback', 100, 600),
                    IntegerParameter('vd_switch', 0, 1)
                ]

dice_sm.levers = [RealParameter('sr', 0.1, 0.5),
            RealParameter('prtp_con', 0.001, 0.015),
            RealParameter('prtp_dam', 0.001, 0.015),
            RealParameter('emdd', -1.0, 2.00),
            IntegerParameter('periodfullpart', 10, 58),
            IntegerParameter('miu_period', 10, 58)
            ]

dice_sm.outcomes = [
            ScalarOutcome('Atmospheric Temperature', kind=ScalarOutcome.MINIMIZE),
            ScalarOutcome('Total Output', kind=ScalarOutcome.INFO),
            ScalarOutcome('Per Capita Consumption', kind=ScalarOutcome.INFO),
            ScalarOutcome('Consumption Growth', kind=ScalarOutcome.INFO),
            ScalarOutcome('Utility of Consumption', kind=ScalarOutcome.MAXIMIZE),
            ScalarOutcome('Per Capita Damage', kind=ScalarOutcome.INFO),
            ScalarOutcome('Damage Growth', kind=ScalarOutcome.INFO),
            ScalarOutcome('Disutility of Damage', kind=ScalarOutcome.MINIMIZE),
            ScalarOutcome('Welfare', kind=ScalarOutcome.MAXIMIZE),
            ScalarOutcome('Undiscounted Period Welfare', kind=ScalarOutcome.INFO),
            ScalarOutcome('Consumption SDR', kind=ScalarOutcome.INFO),
            ScalarOutcome('Damage SDR', kind=ScalarOutcome.INFO)
            ]



In [3]:
from ema_workbench.em_framework.evaluators import BaseEvaluator


In [43]:
with MultiprocessingEvaluator(dice_sm) as evaluator:
    results1 = evaluator.optimize(nfe=5e3, searchover='levers', epsilons=[0.1,]*len(dice_sm.outcomes))

BrokenPipeError: [Errno 32] Broken pipe